Using abcTau to fit ACFs for trials (Figure 2 from the paper). 

Three options to do that:
* use abcTau package for both ACF and fitting
* use ACF calculated before using acf function
* use ACF calculated before using iSTTC concat function

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 

import pickle
import numpy as np
import pandas as pd
from scipy import stats

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

from isttc.scripts.cfg_global import project_folder_path

In [2]:
dataset_folder = project_folder_path + 'synthetic_dataset\\'
results_folder = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau\\'

### Load data (spike trains and calculated acf's)

In [ ]:
with open(dataset_folder + 'trials40.pkl','rb') as f:
    data = pickle.load(f)

trial_dict = data['trial_dict']
alphas = data['alphas']
fr_values = data['fr_values']
taus_ms = data['tau_ms']
n_trials = data['n_trials']
trial_lens = data['trial_lens']

print(f'n spike trains {len(trial_dict)}, trial_lens {trial_lens[0]} ms')

In [5]:
with open(dataset_folder + 'trials40_binned.pkl','rb') as f:
    data_binned = pickle.load(f)

trial_dict_binned = data_binned['trial_dict']
alphas_binned  = data_binned['alphas']
fr_values_binned  = data_binned['fr_values']
taus_ms_binned  = data_binned['tau_ms']
n_trials_binned  = data_binned['n_trials']
trial_lens_binned  = data_binned['trial_lens']

print(f'n spike trains {len(trial_dict_binned)}, trial_lens {trial_lens_binned[0]} ms')

n spike trains 100000, trial_lens 1000 ms


In [ ]:
with open(results_folder + 'tau_pearsonr_trial_50ms_20lags_dict.pkl', "rb") as f:
    pearsonr_trial_avg_dict = pickle.load(f)

with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_dict = pickle.load(f)

print(f'len pearsonr_trial_avg_dict {len(pearsonr_trial_avg_dict)}')
print(f'len sttc_trial_concat_dict {len(sttc_trial_concat_dict)}')

### abcTau for both ACF and fitting

In [4]:
# data parameters
summStat_metric = 'comp_cc'
ifNorm = True # if normalize the autocorrelation or PSD
deltaT = 1 # temporal resolution of data.
binSize = 50 #  bin-size for binning data and computing the autocorrelation.
disp = None # put the dispersion parameter if computed with grid-search
maxTimeLag = 1000 # only used when using autocorrelation for summary statistics
#lm = round(maxTimeLag/binSize) # maximum bin for autocorrelation computation

# desired generative model from the list of 'generative_models.py' and the distance function from 'diatance_functions.py'
generativeModel = 'oneTauOU_gammaSpikes'
distFunc = 'linear_distance'

# Define a uniform prior distribution over the given range
# for a uniform prior: stats.uniform(loc=x_min,scale=x_max-x_min)
t_min = 0.0 # first timescale
t_max = 400.0
priorDist = [stats.uniform(loc= t_min, scale = t_max - t_min)]

# aABC fitting parameters
epsilon_0 = 1  # initial error threshold
min_samples = 100 # min samples from the posterior
steps = 60 # max number of iterations
minAccRate = 0.01 # minimum acceptance rate to stop the iterations
parallel = False # if parallel processing
n_procs = 1 # number of processor for parallel processing (set to 1 if there is no parallel processing)


In [6]:
# 7. Create the model object: Just copy paste the following (this a general definition of the model object, but all parts 
# including generative models, summary statistics computation or distance function can be replaced by your own functions. 
# You can add your handmade functions inside respected modules: "generative_models.py", "distance_functions.py", "summary_stats.py")

# creating model object
class MyModel(abcTau.Model):

    #This method initializes the model object.  
    def __init__(self):
        pass

    # draw samples from the prior. 
    def draw_theta(self):
        theta = []
        for p in self.prior:
            theta.append(p.rvs())
        return theta

    # Choose the generative model (from generative_models)
    # Choose autocorrelation computation method (from basic_functions)
    def generate_data(self, theta):
        # generate synthetic data
        if disp == None:
            syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var)')
        else:
            syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var, disp)')
               
        # compute the summary statistics
        syn_sumStat = abcTau.summary_stats.comp_sumStat(syn_data, summStat_metric, ifNorm, deltaT, binSize, T,\
                                          numBinData, maxTimeLag)   
        return syn_sumStat

    # Computes the summary statistics
    def summary_stats(self, data):
        sum_stat = data
        return sum_stat

    # Choose the method for computing distance (from basic_functions)
    def distance_function(self, data, synth_data):
        if np.nansum(synth_data) <= 0: # in case of all nans return large d to reject the sample
            d = 10**4
        else:
            d = eval('abcTau.distance_functions.' +distFunc + '(data, synth_data)')        
        return d

In [7]:
v = trial_dict_binned[0] 
k = 0

spike_binned = v[0]
numTrials = n_trials_binned[k]
T = trial_lens_binned[k]

numBinData = spike_binned.shape[1]
data_mean = np.mean(spike_binned)
data_var = abcTau.preprocessing.comp_cc(spike_binned, spike_binned, 1, binSize, numBinData)[0]
data_sumStat = abcTau.summary_stats.comp_sumStat(spike_binned, summStat_metric, ifNorm, deltaT, binSize, T, numBinData, maxTimeLag)

In [8]:
data_sumStat

array([ 1.        ,  0.60755511,  0.28021065,  0.06623287, -0.00929441,
       -0.06664008, -0.1502196 , -0.14810597, -0.24018935, -0.1930169 ,
       -0.07217865,  0.01001045,  0.01973116,  0.02658514,  0.03027752,
        0.05051176,  0.03904692,  0.02510819, -0.00332314,  0.        ])

In [11]:
disp = np.var(spike_binned)/ data_mean
print(disp)

5.878139534883721


In [12]:
syn_data, numBinData =  eval('abcTau.generative_models.' + generativeModel + \
                                         '(theta, deltaT, binSize, T, numTrials, data_mean, data_var, disp)')

NameError: name 'theta' is not defined

In [ ]:
results_folder_abctau = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau_abctau\\'
# path and filename to save the intermediate results after running each step
inter_save_direc = results_folder_abctau + 'interim_results\\'
datasave_path = results_folder_abctau + 'final_results\\'

In [ ]:
for k,v in list(trial_dict_binned.items())[0:100]:
    spike_binned = v[0]
    numTrials = n_trials_binned[k]
    T = trial_lens_binned[k]

    numBinData = spike_binned.shape[1]
    data_mean = np.mean(spike_binned)
    data_var = abcTau.preprocessing.comp_cc(spike_binned, spike_binned, 1, binSize, numBinData)[0]
    data_sumStat = abcTau.summary_stats.comp_sumStat(spike_binned, summStat_metric, ifNorm, deltaT, binSize, T, numBinData, maxTimeLag)
    
    # data_sumStat, data_mean, data_var, T, numTrials = abcTau.preprocessing.extract_stats(binary_train, deltaT, binSize,\
    #                                                                                   summStat_metric, ifNorm, maxTimeLag)
    print(f'sumStat len {data_sumStat.shape}, data_mean {data_mean}, data_var {data_var}, T {T}, numTrials {numTrials}, numBinData {numBinData}')

    # Run the aABC algorithm and save the results
    filenameSave = 'spike_train_' + str(k)
    inter_filename = 'spike_train_interim_' + str(k)
    abc_results, final_step = abcTau.fit.fit_withABC(MyModel, data_sumStat, priorDist, inter_save_direc, inter_filename,\
                                                     datasave_path, filenameSave, epsilon_0, min_samples, \
                                                     steps, minAccRate, parallel, n_procs, disp)

### Plotting the results

(took examples from other tutorials - I downloaded them before)

In [ ]:
# load abc results
filename = 'test_steps24'
abc_results = np.load(datasave_path + filename + '.npy', allow_pickle=True)
ind = filename.find('steps') 
final_step = int(filename[ind+5] + filename[ind+6])

# extract estimated parameters
theta_accepted = abc_results[final_step-1]['theta accepted']
tau1 = theta_accepted[0]

# extract the development of the error threshold and acceptance rate during fitting (discarding first iteration)
eps = []
accR = []
steps = np.arange(1,final_step+1)[1:]
for i in range(final_step): 
    step_results = abc_results[i]
    eps.append(step_results['epsilon'])
    accR.append(step_results['n accepted']/step_results['n total'])
eps = eps[1:]
accR = accR[1:]

# ground truth values 
tau1_gt = 150

In [ ]:
np.mean(tau1)

In [ ]:
# plot the results
fig = plt.figure(figsize = (24,6))

# plotting variables
cABC = 'darkorange'
ctheor = 'gray'
cAccR = 'r'
cErr = 'b'
a = 0.5
lw = 3

ax = plt.subplot(141)
ax.hist(tau1, facecolor= cABC, density=True, alpha = a, label = r'Estimated')
sns.kdeplot(tau1, color = cABC)
plt.axvline(tau1_gt, color = ctheor, label = r'Ground truth', linewidth = lw, linestyle = '--')

ax.set_xlabel(r'$\tau_1$ [ms]')
ax.set_ylabel('Probability density')
# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend(frameon = False, bbox_to_anchor=(0.4, 1))

fig.tight_layout()

In [ ]:
sttc_results_folder = 'E:\\work\\q_backup_06_03_2025\\projects\\isttc\\results\\synthetic\\results\\param_fr_alpha_tau\\'

df_all = pd.read_pickle(sttc_results_folder + "summary_tau_all_long_df.pkl")


In [ ]:
df_all[df_all['unit_id'] == 10]